# Configuration Models of Random Hypergraphs
*Phil Chodrow, MIT Operations Research Center and Laboratory for Information and Decision Systems*

This notebook is a basic tutorial for conducting null hypothesis tests by randomizing polyadic data under hypergraph configuration models. The `hypergraph` class is implemented in `hypergraph.py`, and includes methods for some basic descriptives as well as the relevant Monte Carlo algorithms. 

We also illustrate the triadic closure analysis shown in the main text. 

In [1]:
import numpy as np
from hypergraph import *
from matplotlib import pyplot as plt

In [2]:
def read_email_data():
    '''
    Read the email data from the supplied .txt, returning it as a list of tuples. 
    '''
    with open('email-enron.txt', 'r') as f:
        F = f.read()
    L = F.split('\n')
    L = [tuple([int(v) for v in g.split(' ') if v is not '']) for g in L]
    return L

In [3]:
L = read_email_data() # read the data
G = hypergraph(L)     # construct a hypergraph from the data

In [4]:
def print_moments(G):
    D = G.node_degrees()
    K = G.edge_dimensions()
    print('The mean node degree is ' + str(round(D.mean())) + ', and the standard deviation is '+ str(round(np.sqrt(np.var(D)))) + '.')
    print('The mean edge dimension is ' + str(round(K.mean(), 1)) + ', and the standard deviation is '+ str(round(np.sqrt(np.var(K)), 1)) + '.')
    
print_moments(G)

The mean node degree is 188.0, and the standard deviation is 238.0.
The mean edge dimension is 2.5, and the standard deviation is 1.4.


## Monte Carlo Sampling and Triadic Closure

We'll now illustrate the use of Monte Carlo sampling to conduct a simple hypothesis-test using the `hypergraph` class. We'll replicate the triadic closure study shown in the main text. Recall that our metric of interest is the proportion of 3-cycles in the projected graph that are "filled in" by a higher-dimensional edge. 

In [5]:
def count_triangles(C):
    '''
    Count the number of triangles in the (unweighted) projected graph of hypergraph C
    '''
    G = projected_graph(C, as_hyper=False)
    G_ = nx.Graph(G)
    triangles = nx.triangles(G_)
    n_triangles = sum(triangles.values())/3
    return(n_triangles)

def count_filled_triangles(C):
    '''
    Count the number of distinct sub-edges of size 3 in C. 
    '''
    container = set()
    for f in C.C:
        for t in itertools.combinations(f, 3):
            container.add(tuple(sorted(t)))
    return(len(container))

def analyze_closure(G):
    tri = count_triangles(G)
    filled = count_filled_triangles(G)
    print('The rate of simplicial closure in triangles is ' + str(round(filled / tri, 2)) + '.')

In [6]:
analyze_closure(G)

The rate of simplicial closure in triangles is 0.81.


Now we can compare this value to a sample from a null model. Let's start with a stub-labeled null. 

In [7]:
G.MH(n_steps = 100000, label = 'stub')

100000 steps completed.


We can check that the randomized graph has the same degree sequence and edge-dimension sequence as the old: 

In [8]:
print_moments(G)

The mean node degree is 188.0, and the standard deviation is 238.0.
The mean edge dimension is 2.5, and the standard deviation is 1.4.


How does the null value for simplicial closure in triangles compare to the observed value? 

In [9]:
analyze_closure(G)

The rate of simplicial closure in triangles is 0.25.


Simplicial closure in triangles occurs in the observed data three times as often as would be expected by the stub-labeled configuration model. Formally, we should repeatedly randomize and pull many samples, but the spread is not large. Take my word for it, or try it yourself. 

What about the vertex-labeled model? 

The vertex-labeled MH algorithm accepts an additional parameter, `n_clash`. Tracking the counts of each edge is very expensive, so instead we implement an approximation that resamples the edge counts only when a "clash" requires an update. We recommend setting `n_clash = 1` when using vertex-labeled randomization. 

In [10]:
G.MH(n_steps = 100000, label = 'vertex', n_clash = 1)

2319 epochs completed, 100007 steps taken, 473947 steps rejected.


The degree and dimension sequences are again preserved: 

In [11]:
print_moments(G)

The mean node degree is 188.0, and the standard deviation is 238.0.
The mean edge dimension is 2.5, and the standard deviation is 1.4.


What about simplicial closure? 

In [12]:
analyze_closure(G)

The rate of simplicial closure in triangles is 0.21.


# Preserving Node-Edge Dimension Matrices

In some cases, it may be desirable to consider a more restrictive class of nulls that preserve more detailed information about the data. A fairly natural definition is to require that each node preserve not only its total degree, but also the number of edges of each dimension incident to it. Class `hypergraph` implements this as "detailed" MCMC, as we demonstrate here. 

First, we'll read in the data and take a look at the "node-dimension matrix," whose $ij$th entry is the number of edges of dimension $j$ incident on node $i$. 

In [13]:
L = read_email_data() # read the data
G = hypergraph(L)     # construct a hypergraph from the data
A0 = G.node_dimension_matrix().T[:10,]

Now we'll do MCMC, finding that this matrix is preserved when we specify the `detailed = True` in the MCMC call. You can do this with either stub- or vertex-labeling. 

In [14]:
G.MH(n_steps = 100000, label = 'vertex', detailed = True, n_clash = 1)
A1 = G.node_dimension_matrix().T[:10,]

3132 epochs completed, 100011 steps taken, 1330856 steps rejected.


In [15]:
np.mean(A0 == A1) # check that all entries are equal

1.0

# Closing Thoughts

While our particular illustration is quite simple, the recipe is extremely general. To conduct null hypothesis tests, 

1. Measure the empirical value of your quantity of interest on the observed data. 
2. Repeatedly randomize under the appropriate configuration model, measuring the value at appropriate intervals. 
3. Compare the empirical value to the null distribution.  

Some care is required in choosing $n_\mathrm{steps}$ to ensure that the chain is appropriately mixed (i.e. that the hypergraph is close to fully-randomized). To my knowledge, there are no general results on this. A practical approach is to track your quantity of interest over many iterations, and only use samples after they have "leveled off," which suggests stationarity. 